In [ ]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df = pd.read_json("/content/drive/MyDrive/klue-sts-v1.1_train.json")
test = pd.read_json('/content/drive/MyDrive/klue-sts-v1.1_dev.json')

In [ ]:
df['labels'] = df['labels'].map(lambda x: x['binary-label'])
test['labels'] = test['labels'].map(lambda x: x['binary-label'])

In [ ]:
test

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_dev_00000,airbnb-rtt,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.",1,"{'agreement': '0:0:0:0:1:6', 'annotators': ['1..."
1,klue-sts-v1_dev_00001,airbnb-sampled,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,0,"{'agreement': '0:4:3:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_dev_00002,policy-sampled,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,0,"{'agreement': '0:5:2:0:0:0', 'annotators': ['0..."
3,klue-sts-v1_dev_00003,airbnb-rtt,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.",1,"{'agreement': '0:0:0:2:5:0', 'annotators': ['1..."
4,klue-sts-v1_dev_00004,paraKQC-para,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,0,"{'agreement': '1:0:1:3:1:0', 'annotators': ['0..."
...,...,...,...,...,...,...
514,klue-sts-v1_dev_00514,policy-rtt,"문체부는 이를 연차적으로 확대, 시행해 학교운동부와 스포츠클럽 간의 연계를 강화한다.",문화체육관광부는 학교스포츠학과와 스포츠클럽의 연계성을 강화하기 위해 매년 이 프로그...,0,"{'agreement': '0:1:2:2:0:0', 'annotators': ['0..."
515,klue-sts-v1_dev_00515,airbnb-sampled,일단 정확한 정보와 빠른 답변이 정말 좋았습니다.,호스트의 빠른 답변과 유용한 정보들이 정말 좋습니다.,0,"{'agreement': '0:0:1:5:0:0', 'annotators': ['0..."
516,klue-sts-v1_dev_00516,airbnb-sampled,게스트에 대한 배려가 묻어나는 시설들이었습니다.,우선 공간에 대한 센스가 돋보이는 곳이었습니다.,0,"{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
517,klue-sts-v1_dev_00517,policy-sampled,밤하늘을 배경으로 ‘비대면 드론쇼’도 펼쳐진다.,‘비대면 실감형 문화공연 플랫폼’ 개념도.,0,"{'agreement': '5:0:1:0:0:0', 'annotators': ['0..."


In [ ]:
df['sentence'] = [[df['sentence1'][sent],df['sentence2'][sent]] for sent in df.index]

In [ ]:
test['sentence'] = [[test['sentence1'][sent],test['sentence2'][sent]] for sent in test.index]

In [ ]:
# seed
seed = 7777
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 25.9 MB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
     |████████████████████████████████| 6.5 MB 65.2 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 72.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


In [ ]:
print(f"Shape: {df.shape}\nLabel : {df.labels.value_counts()}")

Shape: (11668, 7)
Label : 0    6066
1    5602
Name: labels, dtype: int64


# 데이터셋 만들기

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split

In [ ]:
class CustomDataset(Dataset):
  """
  - input_data: list of string
  - target_data: list of int
  """

  def __init__(self, input_data:list, target_data:list) -> None:
      self.X = input_data
      self.Y = target_data

  def __len__(self):
      return len(self.X)

  def __getitem__(self, index):
      return self.X[index], self.Y[index]

In [ ]:
dataset = CustomDataset(df['sentence'], df.labels)

In [ ]:
testset = CustomDataset(test['sentence'], test.labels)

In [ ]:
# map-stype dataset 클래스는 indexing이 가능함
dataset[1]

(['위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.',
  '시민들 스스로 자발적인 예방 노력을\xa0한 것은 아산 뿐만이 아니었다.'],
 0)

In [ ]:
# 데이터 셋 총 개수 확인 가능
len(dataset)

11668

In [ ]:
n_train = int(len(dataset)*0.9)
n_valid = int(len(dataset)*0.1)

In [ ]:
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid+1])

In [ ]:
len(train_dataset)

10501

In [ ]:
len(valid_dataset)

1167

In [ ]:
!pip install transformers

###토큰화


In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
def custom_collate_fn(batch):
  """
  - batch: list of tuples (input_data(string),et_data(int)) targ
  
  한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
  이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
  토큰 개수는 배치 내 가장 긴 문장으로 해야함.
  또한 최대 길이를 넘는 문장은 최대 길이 이후의 토큰을 제거하도록 해야 함
  토크나이즈된 결과 값은 텐서 형태로 반환하도록 해야 함
  
  한 배치 내 레이블(target)은 텐서화 함.
  
  (input, target) 튜플 형태를 반환.
  """
  global tokenizer_bert
  
  input_list, target_list =[i for i,z in batch], [z for i,z in batch]
  
  tensorized_input = tokenizer_bert(
      input_list,
      return_tensors='pt',
      padding='longest'
  )
  
  tensorized_label = torch.tensor(target_list)
  
  return (tensorized_input, tensorized_label)
    

### 위에서 구현한 `custom_collate_fn`함수를 적용해 DataLoader 인스턴스 생성
- `train_dataloader`
    - batch_size = 32
    - collate_fn = 위에서 구현한 함수
    - sampler = `RandomSampler()`
        - `train_dataset`의 학습 데이터를 셔플링 함
- `valid_dataloader`
    - batch_size = 64
    - collate_fn = 위에서 구현한 함수
    - sampler = `SequentialSampler()`
        - `valid_dataset`의 검증 데이터를 순차적으로 정렬함 (셔플X)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size= 32, sampler=RandomSampler(train_dataset), collate_fn =custom_collate_fn )

valid_dataloader = DataLoader(valid_dataset, batch_size= 64, sampler=SequentialSampler(valid_dataset), collate_fn =custom_collate_fn )

In [ ]:
test_dataloader= DataLoader(testset, batch_size= 32, sampler=SequentialSampler(testset), collate_fn =custom_collate_fn )

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.nn.utils import clip_grad_norm_
from transformers import get_linear_schedule_with_warmup, get_constant_schedule

In [ ]:
def save_checkpoint(path, model, optimizer, scheduler, epoch, loss):
    file_name = f'{path}/model.ckpt.{epoch}'
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss' : loss
        }, 
        file_name
    )
    
    print(f"Saving epoch {epoch} checkpoint at {file_name}")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CustomClassifier(nn.Module):

  def __init__(self, hidden_size: int, n_label: int):
    super(CustomClassifier, self).__init__()

    self.bert = BertModel.from_pretrained("klue/bert-base")

    dropout_rate = 0.1
    linear_layer_hidden_size = 32

    self.classifier =  nn.Sequential(
          nn.Linear(hidden_size,linear_layer_hidden_size),
          nn.ReLU(),
          nn.Dropout(dropout_rate),
          nn.Linear(linear_layer_hidden_size,2)
        )
  
  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
    outputs = self.bert(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
    )
    
    # BERT 모델의 마지막 레이어의 첫번재 토큰을 인덱싱
    cls_token_last_hidden_states = outputs['pooler_output'] # 마지막 layer의 첫 번째 토큰 ("[CLS]") 벡터를 가져오기, shape = (1, hidden_size)
    logits = self.classifier(cls_token_last_hidden_states)

    return logits

In [ ]:
def initializer(train_dataloader, epochs=2):
    """
    모델, 옵티마이저, 스케쥴러 초기화
    """
    
    model = CustomClassifier(hidden_size=768, n_label=2)

    optimizer = AdamW(
        model.parameters(), # update 대상 파라미터를 입력
        lr=2e-5,
        eps=1e-8
    )
    
    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps = 0, # 여기서는 warmup을 사용하지 않는다.
        num_training_steps = total_steps
    )

    return model, optimizer, scheduler

# 새 섹션

In [ ]:
loss_fct = CrossEntropyLoss()
def validate(model, valid_dataloader):
   
    # 모델을 evaluate 모드로 설정 & device 할당
    model.eval()
    model.to(device)
    
    total_loss, total_acc= 0,0
        
    for step, batch in enumerate(valid_dataloader):
        
        # tensor 연산 전, 각 tensor에 device 할당
        batch = tuple(item.to(device) for item in batch)
            
        batch_input, batch_label = batch
            
        # gradient 계산하지 않음
        with torch.no_grad():
            logits = model(**batch_input)
            
        # loss
        loss = loss_fct(logits, batch_label)
        total_loss += loss.item()
        
        # accuracy
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1).flatten()
        acc = (preds == batch_label).cpu().numpy().mean()
        total_acc+=acc
    
    total_loss = total_loss/(step+1)
    total_acc = total_acc/(step+1)*100

    return total_loss, total_acc

NameError: ignored

In [ ]:
loss_fct = CrossEntropyLoss()

def train(model, train_dataloader, valid_dataloader=None, epochs=2):
        
        # train_dataloaer 학습을 epochs만큼 반복
        for epoch in range(epochs):
            print(f"*****Epoch {epoch} Train Start*****")
            
            # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
            total_loss, batch_loss, batch_count = 0,0,0
        
            # model을 train 모드로 설정 & device 할당
            model.train()
            model.to(device)
            
            # data iterator를 돌면서 하나씩 학습
            for step, batch in enumerate(train_dataloader):
                batch_count+=1
                
                # tensor 연산 전, 각 tensor에 device 할당
                batch = tuple(item.to(device) for item in batch)
            
                batch_input, batch_label = batch
            
                # batch마다 모델이 갖고 있는 기존 gradient를 초기화
                model.zero_grad()
            
                # forward
                logits = model(**batch_input)
            
                # loss
                loss = loss_fct(logits, batch_label)
                batch_loss += loss.item()
                total_loss += loss.item()
            
                # backward -> 파라미터의 미분(gradient)를 자동으로 계산
                loss.backward()
                
                # gradient clipping 적용 
                clip_grad_norm_(model.parameters(), 1.0)
                
                # optimizer & scheduler 업데이트
                optimizer.step()
                scheduler.step()
                
                # 배치 10개씩 처리할 때마다 평균 loss와 lr를 출력
                if (step % 10 == 0 and step != 0):
                    learning_rate = optimizer.param_groups[0]['lr']
                    print(f"Epoch: {epoch}, Step : {step}, LR : {learning_rate}, Avg Loss : {batch_loss / batch_count:.4f}")

                    # reset 
                    batch_loss, batch_count = 0,0

            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finish*****\n")
            
            if valid_dataloader is not None:
                print(f"*****Epoch {epoch} Valid Start*****")
                valid_loss, valid_acc = validate(model, valid_dataloader)
                print(f"Epoch {epoch} Valid Loss : {valid_loss:.4f} Valid Acc : {valid_acc:.2f}")
                print(f"*****Epoch {epoch} Valid Finish*****\n")
            
            # checkpoint 저장
            save_checkpoint(".", model, optimizer, scheduler, epoch, total_loss/(step+1))
                
        print("Train Completed. End Program.")

In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from numpy.core.fromnumeric import nonzero
from tqdm import tqdm, trange

In [ ]:
epochs=4
model, optimizer, scheduler = initializer(train_dataloader, epochs)
train(model, train_dataloader, valid_dataloader, epochs)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total train steps with 4 epochs: 1316
*****Epoch 0 Train Start*****
Epoch: 0, Step : 10, LR : 1.9832826747720366e-05, Avg Loss : 0.5556
Epoch: 0, Step : 20, LR : 1.968085106382979e-05, Avg Loss : 0.3910
Epoch: 0, Step : 30, LR : 1.952887537993921e-05, Avg Loss : 0.2656
Epoch: 0, Step : 40, LR : 1.9376899696048635e-05, Avg Loss : 0.2678
Epoch: 0, Step : 50, LR : 1.922492401215806e-05, Avg Loss : 0.1713
Epoch: 0, Step : 60, LR : 1.9072948328267476e-05, Avg Loss : 0.2239
Epoch: 0, Step : 70, LR : 1.89209726443769e-05, Avg Loss : 0.1611
Epoch: 0, Step : 80, LR : 1.8768996960486324e-05, Avg Loss : 0.2123
Epoch: 0, Step : 90, LR : 1.8617021276595745e-05, Avg Loss : 0.1657
Epoch: 0, Step : 100, LR : 1.846504559270517e-05, Avg Loss : 0.1987
Epoch: 0, Step : 110, LR : 1.8313069908814593e-05, Avg Loss : 0.1532
Epoch: 0, Step : 120, LR : 1.8161094224924014e-05, Avg Loss : 0.1877
Epoch: 0, Step : 130, LR : 1.8009118541033438e-05, Avg Loss : 0.1921
Epoch: 0, Step : 140, LR : 1.785714285714286e-05, 

In [ ]:
checkpoint = torch.load('model.ckpt.1')

In [ ]:
epochs=1
model, optimizer, scheduler = initializer(train_dataloader, epochs)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total train steps with 1 epochs: 329


In [ ]:
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [ ]:
def predict(model, test_dataloader):
    """
    test_dataloader의 label별 확률값과 실제 label 값을 반환
    """

    # model을 eval 모드로 설정 & device 할당
    model.eval()
    model.to(device)

    all_logits = []
    all_labels = []

    for step, batch in enumerate(test_dataloader):
        
        batch_input, batch_label = batch
        
        # batch_input을 device 할당
        batch_input.to(device)

        # model에 batch_input을 넣어 logit 반환 & all_logits, all_labels 리스트에 값 추가 
        with torch.no_grad():
            logits = model(**batch_input)
        all_logits.append(logits)
        all_labels.append(batch_label)
    

    probs = F.softmax(torch.cat(all_logits), dim =1).cpu().numpy() # logits을 확률값으로 변환 & Tensor 타입을 numpy.array 타입으로 변환
    all_labels = torch.cat(all_labels).numpy() #  Tensor 타입을 numpy.array 타입으로 변환

    return probs, all_labels

In [ ]:
def accuracy(probs, labels):
    y_pred = np.where(probs < 0.5, probs, 1)
    y_pred = np.where(probs >= 0.5, y_pred, 0) # probs(확률값)을 label로 변경(0.5 이상이면 1, 0.5 미만이면 0)
    acc =   float(sum(labels == np.argmax(y_pred, axis=1)) / len(labels))
    return acc 

In [ ]:
probs, labels = predict(model, test_dataloader)

In [ ]:
accuracy(probs, labels)

0.7957610789980732

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score,f1_score

In [ ]:
accuracy_score(labels,  np.argmax(probs, axis=1))

0.7957610789980732

In [ ]:
roc_auc_score(labels, probs[:,1])

0.891243539069626

In [ ]:
f1_score(labels,  np.argmax(probs, axis=1))

0.7984790874524715